# Analysis of the intergenerational transmission of skills : Construction of the database (Fynch MEYNENT) #

The aim of this notebook is to allow to create the database I used in my report about my analysis of the intergenerational transmission of skills. We will merge a part of a reworked O*NET Database with an extract of the NEPS database to obtain, for each child's outcome in skills, a level of skills from their parents estimated by their occupational skills. 

Some of the files I used are on: https://github.com/jmeynent/TrinityResearch

Firstly, let's install and import the packages we need. 

In [1]:
!pip install tslearn
!pip install h5py
!pip install kneed
!pip install openpyxl
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np
import pandas as pd

It's possible to find the O*NET Skills database in the following link. You have to download it in the excel format. It's also in the GitHub link I mentioned above. 
https://www.onetcenter.org/dictionary/27.3/excel/skills.htm

However, the NEPS database is not in free access. If you find one, remind to convert it in an excel format to be sure that it will work. 

In [3]:
#Importation of the databases
onet = pd.read_excel('Skills.xlsx')  #Replace Skills if the name is different
neps = pd.read_excel('insert_name.xls') #Replace insert_name if the name is different, and adapt the format

## Construction of the occupational skills score ##

In [4]:
onet #Just to have a view on the database. It's not necessary to construct. 

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8.0,0.16,4.43,5.07,N,N,07/2014,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.88,8.0,0.23,4.43,5.32,N,N,07/2014,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.00,8.0,0.00,4.00,4.00,N,NaN,07/2014,Analyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61105,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.b,Management of Financial Resources,LV,Level,1.12,8.0,0.13,0.88,1.37,N,N,08/2019,Analyst
61106,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,IM,Importance,2.00,8.0,0.00,2.00,2.00,N,NaN,08/2019,Analyst
61107,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,LV,Level,1.88,8.0,0.13,1.63,2.12,N,N,08/2019,Analyst
61108,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.d,Management of Personnel Resources,IM,Importance,2.88,8.0,0.13,2.63,3.12,N,NaN,08/2019,Analyst


### Creation of a crosswalk file ###

To work with the NEPS, we need to convert the O*NET-SOC Code into ISCO-08 Code. Firstly, we import the crosswalk files. We will convert O*NET SOC Code 2019 into SOC code 2018, then convert SOC code 2018 into SOC code 2010, and finally convert SOC code 2010 to ISCO08.
Firstly, we import the crosswalk files (they are available on my github)

In [5]:
onetsoc18 = pd.read_excel('ONET_to_Soc18.xlsx')
soc18soc10 = pd.read_excel('Soc18_to_Soc10.xlsx')
soc10isco08 = pd.read_excel('Soc10_to_ISCO08.xls')

Then, we construct a crosswalk to obtain a crosswalk files from O*NET-Soc Code 2019 to ISco-08 Code.

In [6]:
onetsoc18.rename(columns={'O*NET-SOC 2019 Occupation Listings' : 'ONET19', 'Unnamed: 2' : 'SOC18'}, inplace=True) #Rename columns to make it more easy to understand
onetsoc18.drop(['Unnamed: 1', 'Unnamed: 3'], axis=1, inplace=True) #Delete useless columns
onetsoc18.drop([0,1,2], axis=0, inplace=True) #Delete useless rows
onetsoc18 #Our final crosswalk file from ONET-2019 to SOC-2018.

,ONET19,SOC18
3,11-1011.00,11-1011
4,11-1011.03,11-1011
5,11-1021.00,11-1021
6,11-1031.00,11-1031
7,11-2011.00,11-2011
...,...,...
1014,55-3014.00,55-3014
1015,55-3015.00,55-3015
1016,55-3016.00,55-3016
1017,55-3018.00,55-3018


In [7]:
soc18soc10.rename(columns={'U.S. Bureau of Labor Statistics' : 'SOC10', 'Unnamed: 2' : 'SOC18'}, inplace=True) #Rename
soc18soc10.drop(['Unnamed: 1', 'Unnamed: 3'], axis=1, inplace=True) #Delete useless columns
soc18soc10.drop([0,1,2,3,4,5,6,7], axis=0, inplace=True) #Delete useless rows
soc18soc10

,SOC10,SOC18
8,11-1011,11-1011
9,11-1021,11-1021
10,11-1031,11-1031
11,11-2011,11-2011
12,11-2021,11-2021
...,...,...
903,55-3015,55-3015
904,55-3016,55-3016
905,55-3017,17-3029
906,55-3018,55-3018


In [8]:
soc10isco08.rename(columns={'soc10' : 'SOC10'}, inplace=True)
soc10isco08

,SOC10,isco08
0,111011,1112
1,111011,1113
2,111011,1120
3,111021,1112
4,111021,1114
...,...,...
1126,553015,310
1127,553016,310
1128,553017,310
1129,553018,310


In [9]:
#We convert SOC10 column of one of the crosswalk to allow a merge with the other
soc18soc10['SOC10'] = soc18soc10['SOC10'].replace({'-': ''}, regex=True)

In [10]:
crosswalk = onetsoc18.merge(soc18soc10, on='SOC18') #We obtain the crosswalk ONET19 to SOC10
crosswalk['SOC10'] =  crosswalk['SOC10'].astype(int) #We convert the type of the column to make the joint possible
crosswalk = crosswalk.merge(soc10isco08, on='SOC10') #We obtain the crosswalk ONET19 to ISC1O08

In [11]:
#We delete the useless columns
crosswalk.drop(['SOC18', 'SOC10'], axis=1, inplace=True)

#we give a size of 4 characters on the ISCO code to allow future joints. 
crosswalk['isco08'] = crosswalk['isco08'].astype(str).str.zfill(4)
crosswalk

,ONET19,isco08
0,11-1011.00,1112
1,11-1011.00,1113
2,11-1011.00,1120
3,11-1011.03,1112
4,11-1011.03,1113
...,...,...
1595,55-3014.00,0310
1596,55-3015.00,0310
1597,55-3016.00,0310
1598,55-3018.00,0310


We have our crosswalk file. Now, we had it to database


In [12]:
onet = onet.merge(crosswalk, left_on='O*NET-SOC Code', right_on='ONET19', how='left')
onet

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source,ONET19,isco08
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst,11-1011.00,1112
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst,11-1011.00,1113
2,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8.0,0.13,3.88,4.37,N,NaN,07/2014,Analyst,11-1011.00,1120
3,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8.0,0.16,4.43,5.07,N,N,07/2014,Analyst,11-1011.00,1112
4,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8.0,0.16,4.43,5.07,N,N,07/2014,Analyst,11-1011.00,1113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89665,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.b,Management of Financial Resources,LV,Level,1.12,8.0,0.13,0.88,1.37,N,N,08/2019,Analyst,53-7121.00,9333
89666,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,IM,Importance,2.00,8.0,0.00,2.00,2.00,N,NaN,08/2019,Analyst,53-7121.00,9333
89667,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.c,Management of Material Resources,LV,Level,1.88,8.0,0.13,1.63,2.12,N,N,08/2019,Analyst,53-7121.00,9333
89668,53-7121.00,"Tank Car, Truck, and Ship Loaders",2.B.5.d,Management of Personnel Resources,IM,Importance,2.88,8.0,0.13,2.63,3.12,N,NaN,08/2019,Analyst,53-7121.00,9333


### Reshaping data ###

We have to separate datas indexed by level and those index by importance. 

In [13]:
skill_im = onet[onet['Scale ID']=='IM']
skill_lvl = onet[onet['Scale ID']=='LV']

We will reshape the database in order to obtain, for each ISCO occupational level, the value of each kind of competence. Because there's less ISCO code than O*NET one, we aggregate the value by tooking the average. 

In [14]:
pivot_lvl = skill_lvl.pivot_table(index='isco08', columns='Element Name', values='Data Value', aggfunc='mean')
pivot_lvl #Just to see how does it look

Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,Judgment and Decision Making,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
isco08,,,,,,,,,,,,,,,,,,,,,
0310,3.375000,3.440000,3.310000,3.060000,3.560000,2.935000,2.935000,1.810,3.000000,3.190000,...,2.500000,2.565000,2.940000,3.120000,3.000000,3.060000,1.940000,3.060000,3.190000,3.185000
1112,4.157500,4.220000,4.530000,4.440000,4.312500,0.000000,0.217500,0.000,3.565000,4.437500,...,1.435000,3.687500,4.002500,4.312500,4.125000,4.060000,0.902500,4.065000,0.595000,4.190000
1113,4.315000,4.440000,4.560000,4.500000,4.435000,0.000000,0.435000,0.000,3.630000,4.875000,...,1.500000,3.315000,4.065000,4.500000,4.690000,4.560000,0.935000,4.315000,0.000000,4.315000
1114,3.778889,3.887778,3.541111,3.847778,4.068889,0.347778,0.527778,0.000,3.444444,3.680000,...,1.164444,3.234444,3.748889,3.998889,3.362222,3.402222,1.178889,3.568889,1.181111,3.902222
1120,4.083333,4.293333,4.290000,4.293333,4.290000,0.000000,0.290000,0.000,3.503333,4.500000,...,1.373333,3.250000,4.043333,4.333333,4.126667,4.080000,0.996667,4.126667,0.460000,4.170000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9621,2.185000,3.000000,2.250000,2.565000,2.685000,0.875000,0.310000,0.000,2.060000,2.125000,...,0.060000,2.810000,2.380000,2.620000,1.940000,1.870000,0.315000,2.500000,1.625000,2.435000
9622,2.873333,2.916667,2.666667,3.000000,2.996667,3.460000,2.753333,2.830,2.333333,2.833333,...,1.080000,2.210000,2.290000,2.793333,2.456667,2.376667,1.333333,2.793333,3.540000,2.626667
9623,2.120000,2.940000,2.685000,2.370000,2.815000,2.060000,1.620000,0.935,2.000000,2.560000,...,0.370000,2.685000,2.310000,2.810000,2.315000,2.000000,0.310000,2.685000,2.440000,2.625000


In [15]:
#We do the same for importance level
pivot_imp =  skill_im.pivot_table(index='isco08', columns='Element Name', values='Data Value', aggfunc='mean')
pivot_imp

Element Name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,Judgment and Decision Making,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
isco08,,,,,,,,,,,,,,,,,,,,,
0310,3.435000,3.440000,3.250000,2.815000,3.435000,3.065000,2.685000,2.000000,2.870000,3.185000,...,2.565000,2.620000,2.815000,3.185000,2.940000,3.000000,2.190000,3.000000,3.125000,3.185000
1112,3.750000,4.030000,4.032500,4.000000,4.095000,1.000000,1.060000,1.000000,3.310000,4.000000,...,2.000000,3.435000,3.970000,4.125000,3.687500,3.595000,1.782500,3.720000,1.470000,3.842500
1113,3.875000,4.060000,4.190000,4.000000,4.250000,1.000000,1.120000,1.000000,3.185000,4.190000,...,2.000000,3.185000,4.065000,4.190000,4.000000,4.000000,1.815000,3.690000,1.000000,4.060000
1114,3.432222,3.876667,3.486667,3.804444,3.891111,1.193333,1.362222,1.000000,3.264444,3.625556,...,1.820000,3.224444,3.611111,3.946667,3.277778,3.262222,1.834444,3.458889,1.805556,3.667778
1120,3.750000,4.040000,3.960000,4.000000,4.126667,1.000000,1.080000,1.000000,3.163333,3.960000,...,1.960000,3.206667,4.043333,4.126667,3.666667,3.666667,1.836667,3.710000,1.333333,3.790000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9621,2.435000,3.435000,2.750000,2.875000,3.000000,1.685000,1.310000,1.000000,2.190000,2.815000,...,1.060000,3.375000,2.815000,3.435000,2.060000,2.000000,1.315000,3.065000,1.940000,2.875000
9622,2.953333,3.040000,2.953333,3.000000,3.206667,3.373333,2.876667,2.793333,2.373333,3.000000,...,1.706667,2.546667,2.750000,3.000000,2.540000,2.416667,1.873333,2.876667,3.496667,2.666667
9623,2.685000,2.940000,2.880000,2.620000,2.940000,2.690000,2.060000,1.620000,2.190000,2.750000,...,1.370000,2.940000,2.685000,2.940000,2.440000,2.375000,1.250000,2.940000,2.875000,2.620000


### Principal Factor Analysis ###

The future objective is to make a PFA on the competences. We will do it with Stata. Therefore, we have to export it in dta format. We also conserve a copy in excel format

In [17]:
pivot_lvl.to_stata('level_pivot.dta')
pivot_lvl.to_excel('level_pivot.xlsx')
pivot_imp.to_stata('imp_pivot.dta')
pivot_imp.to_excel('imp_pivot.xlsx')

/tmp/ipykernel_425/553118153.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Active Learning   ->   Active_Learning
    Active Listening   ->   Active_Listening
    Complex Problem Solving   ->   Complex_Problem_Solving
    Critical Thinking   ->   Critical_Thinking
    Equipment Maintenance   ->   Equipment_Maintenance
    Equipment Selection   ->   Equipment_Selection
    Judgment and Decision Making   ->   Judgment_and_Decision_Making
    Learning Strategies   ->   Learning_Strategies
    Management of Financial Resources   ->   Management_of_Financial_Resource
    Management of Material Resources   ->   Management_of_Material_Resources
    Management of Personnel Resources   ->   Management_of_Personnel_Resource
    Operation and Control   ->   Operation_and_Control
    Operations Analysis   ->   Operations_Analysis
    Operations Monitoring   ->   Operations_Monitoring
    Quality Control Analys

Then, run on Stata the PFA by using the PFA.do file available on the github I gave. 

### Finish the occupational skills database ###

In [26]:
#We import what we obtained from Stata
lvl = pd.read_excel('PFA_lvl.xls')
imp = pd.read_excel('PFA_imp.xls')

In [27]:
imp

,isco08,Active_Learning,Active_Listening,Complex_Problem_Solving,Coordination,Critical_Thinking,Equipment_Maintenance,Equipment_Selection,Installation,Instructing,...,Systems_Analysis,Systems_Evaluation,Technology_Design,Time_Management,Troubleshooting,Writing,f1,f2,f3,f4
0,310,3.435000,3.440000,3.250000,2.815000,3.435000,3.065000,2.685000,2.000000,2.870000,...,2.940000,3.000000,2.190000,3.000000,3.125000,3.185000,0.254217,1.677829,1.162194,-0.887223
1,1112,3.750000,4.030000,4.032500,4.000000,4.095000,1.000000,1.060000,1.000000,3.310000,...,3.687500,3.595000,1.782500,3.720000,1.470000,3.842500,1.341973,-0.569583,0.192803,2.371664
2,1113,3.875000,4.060000,4.190000,4.000000,4.250000,1.000000,1.120000,1.000000,3.185000,...,4.000000,4.000000,1.815000,3.690000,1.000000,4.060000,1.160001,-0.805663,0.654340,2.738459
3,1114,3.432222,3.876667,3.486667,3.804444,3.891111,1.193333,1.362222,1.000000,3.264444,...,3.277778,3.262222,1.834444,3.458889,1.805556,3.667778,0.991501,-0.319348,-0.006640,1.456851
4,1120,3.750000,4.040000,3.960000,4.000000,4.126667,1.000000,1.080000,1.000000,3.163333,...,3.666667,3.666667,1.836667,3.710000,1.333333,3.790000,1.115307,-0.635790,0.261679,2.741052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,9621,2.435000,3.435000,2.750000,2.875000,3.000000,1.685000,1.310000,1.000000,2.190000,...,2.060000,2.000000,1.315000,3.065000,1.940000,2.875000,-0.781488,-0.570138,-0.953864,-0.232578
416,9622,2.953333,3.040000,2.953333,3.000000,3.206667,3.373333,2.876667,2.793333,2.373333,...,2.540000,2.416667,1.873333,2.876667,3.496667,2.666667,-0.041227,2.122485,0.109372,-0.720609
417,9623,2.685000,2.940000,2.880000,2.620000,2.940000,2.690000,2.060000,1.620000,2.190000,...,2.440000,2.375000,1.250000,2.940000,2.875000,2.620000,-0.869004,0.786314,-0.365761,-0.122075
418,9624,2.310000,2.935000,2.380000,2.750000,2.815000,2.315000,2.125000,1.190000,2.120000,...,1.880000,1.880000,1.375000,2.560000,2.375000,2.380000,-1.300712,0.422395,-0.976066,-0.261673


In [28]:
lvl

,isco08,Active_Learning,Active_Listening,Complex_Problem_Solving,Coordination,Critical_Thinking,Equipment_Maintenance,Equipment_Selection,Installation,Instructing,...,Systems_Analysis,Systems_Evaluation,Technology_Design,Time_Management,Troubleshooting,Writing,f1,f2,f3,f4
0,310,3.375000,3.440000,3.310000,3.060000,3.560000,2.935000,2.935000,1.810,3.000000,...,3.000000,3.060000,1.940000,3.060000,3.190000,3.185000,0.634280,1.851304,0.645326,-1.073805
1,1112,4.157500,4.220000,4.530000,4.440000,4.312500,0.000000,0.217500,0.000,3.565000,...,4.125000,4.060000,0.902500,4.065000,0.595000,4.190000,1.603612,-0.448209,-0.434995,2.328412
2,1113,4.315000,4.440000,4.560000,4.500000,4.435000,0.000000,0.435000,0.000,3.630000,...,4.690000,4.560000,0.935000,4.315000,0.000000,4.315000,1.771290,-0.593715,-0.273366,2.884460
3,1114,3.778889,3.887778,3.541111,3.847778,4.068889,0.347778,0.527778,0.000,3.444444,...,3.362222,3.402222,1.178889,3.568889,1.181111,3.902222,0.953207,-0.302851,-0.288154,1.458356
4,1120,4.083333,4.293333,4.290000,4.293333,4.290000,0.000000,0.290000,0.000,3.503333,...,4.126667,4.080000,0.996667,4.126667,0.460000,4.170000,1.440997,-0.514948,-0.315793,2.697041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,9621,2.185000,3.000000,2.250000,2.565000,2.685000,0.875000,0.310000,0.000,2.060000,...,1.940000,1.870000,0.315000,2.500000,1.625000,2.435000,-1.308836,-0.560847,-0.379486,-0.170033
416,9622,2.873333,2.916667,2.666667,3.000000,2.996667,3.460000,2.753333,2.830,2.333333,...,2.456667,2.376667,1.333333,2.793333,3.540000,2.626667,-0.131781,2.100968,0.061713,-0.983449
417,9623,2.120000,2.940000,2.685000,2.370000,2.815000,2.060000,1.620000,0.935,2.000000,...,2.315000,2.000000,0.310000,2.685000,2.440000,2.625000,-0.820915,0.608166,-0.208435,-0.472548
418,9624,2.125000,2.630000,2.315000,2.500000,2.565000,2.370000,1.750000,0.250,1.940000,...,1.625000,1.500000,0.500000,2.310000,2.315000,2.250000,-1.292948,0.539814,-0.575147,-0.359435


Now, we will delete useless columns and rename useful ones. We also uniformize the format of isco08 code to allow merge. 

In [29]:
imp["isco08"] = imp["isco08"].astype(str).str.zfill(4)

In [30]:
imp = imp[['isco08', 'f1', 'f2', 'f3', 'f4']] #We only conserve principal factors
imp.rename(columns = {'f1' : 'IM_cog_deci_skills', 'f2': 'IM_manual_skills', 'f3' : 'IM_comp_skills', 'f4' : 'IM_manag_skills'}, inplace=True)
imp

/tmp/ipykernel_425/4176696089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imp.rename(columns = {'f1' : 'IM_cog_deci_skills', 'f2': 'IM_manual_skills', 'f3' : 'IM_comp_skills', 'f4' : 'IM_manag_skills'}, inplace=True)


,isco08,IM_cog_deci_skills,IM_manual_skills,IM_comp_skills,IM_manag_skills
0,0310,0.254217,1.677829,1.162194,-0.887223
1,1112,1.341973,-0.569583,0.192803,2.371664
2,1113,1.160001,-0.805663,0.654340,2.738459
3,1114,0.991501,-0.319348,-0.006640,1.456851
4,1120,1.115307,-0.635790,0.261679,2.741052
...,...,...,...,...,...
415,9621,-0.781488,-0.570138,-0.953864,-0.232578
416,9622,-0.041227,2.122485,0.109372,-0.720609
417,9623,-0.869004,0.786314,-0.365761,-0.122075
418,9624,-1.300712,0.422395,-0.976066,-0.261673


In [31]:
#We do the same for level database
lvl["isco08"] = lvl["isco08"].astype(str).str.zfill(4)
lvl = lvl[['isco08', 'f1', 'f2', 'f3', 'f4']]
lvl.rename(columns = {'f1' : 'LV_cog_deci_skills', 'f2': 'LV_manual_skills', 'f3' : 'LV_comp_skills', 'f4' : 'LV_manag_skills'}, inplace=True)
lvl

/tmp/ipykernel_425/581462118.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lvl.rename(columns = {'f1' : 'LV_cog_deci_skills', 'f2': 'LV_manual_skills', 'f3' : 'LV_comp_skills', 'f4' : 'LV_manag_skills'}, inplace=True)


,isco08,LV_cog_deci_skills,LV_manual_skills,LV_comp_skills,LV_manag_skills
0,0310,0.634280,1.851304,0.645326,-1.073805
1,1112,1.603612,-0.448209,-0.434995,2.328412
2,1113,1.771290,-0.593715,-0.273366,2.884460
3,1114,0.953207,-0.302851,-0.288154,1.458356
4,1120,1.440997,-0.514948,-0.315793,2.697041
...,...,...,...,...,...
415,9621,-1.308836,-0.560847,-0.379486,-0.170033
416,9622,-0.131781,2.100968,0.061713,-0.983449
417,9623,-0.820915,0.608166,-0.208435,-0.472548
418,9624,-1.292948,0.539814,-0.575147,-0.359435


In [32]:
#we merge it to have level and importance in the same dataframe
job_skill = pd.merge(imp, lvl, on='isco08')
job_skill

,isco08,IM_cog_deci_skills,IM_manual_skills,IM_comp_skills,IM_manag_skills,LV_cog_deci_skills,LV_manual_skills,LV_comp_skills,LV_manag_skills
0,0310,0.254217,1.677829,1.162194,-0.887223,0.634280,1.851304,0.645326,-1.073805
1,1112,1.341973,-0.569583,0.192803,2.371664,1.603612,-0.448209,-0.434995,2.328412
2,1113,1.160001,-0.805663,0.654340,2.738459,1.771290,-0.593715,-0.273366,2.884460
3,1114,0.991501,-0.319348,-0.006640,1.456851,0.953207,-0.302851,-0.288154,1.458356
4,1120,1.115307,-0.635790,0.261679,2.741052,1.440997,-0.514948,-0.315793,2.697041
...,...,...,...,...,...,...,...,...,...
415,9621,-0.781488,-0.570138,-0.953864,-0.232578,-1.308836,-0.560847,-0.379486,-0.170033
416,9622,-0.041227,2.122485,0.109372,-0.720609,-0.131781,2.100968,0.061713,-0.983449
417,9623,-0.869004,0.786314,-0.365761,-0.122075,-0.820915,0.608166,-0.208435,-0.472548
418,9624,-1.300712,0.422395,-0.976066,-0.261673,-1.292948,0.539814,-0.575147,-0.359435


### Work on the NEPS and joints ###

In [34]:
neps

,ID_t,PARENTS_EDUCATION,meduYRS,feduYRS,PARENTS_ISEI,mISEI,fISEI,PARENTS_ISCO,mISCO_88,mISCO_08,...,dgn7_sc3b,dgn10_sc3b,COGNITIVE_MATH,man5_sc1,man7_sc1,man9_sc1,COGNITIVE_SCIENCE,scn6_sc1,scn8_sc1,scn10_sc1
0,8054956,1,16.0,16.0,1,79.489998,62.130001,1,2140.0,2100.0,...,10,10,1.0,-.7984647,.1717753,-.6099,1.0,1.36372,1.85782,.72872
1,8054957,1,18.0,16.0,1,73.910004,74.660004,1,2419.0,2432.0,...,not participated,not participated,1.0,-2.787582,not participated,not participated,1.0,-1.81285,-2.01072,not participated
2,8054966,1,16.0,13.0,1,70.500000,54.549999,1,2446.0,2635.0,...,2,7,1.0,-.7828258,-.0418512,.2241,1.0,-1.41709,-.72219,-.21941
3,8054975,1,13.0,13.0,1,24.530001,24.530001,1,5122.0,5120.0,...,4,10,1.0,-.0086852,.6254676,.5545,1.0,-.22023,-.20923,.43187
4,8054979,1,15.0,15.0,1,31.080000,51.560001,1,5141.0,5142.0,...,4,not participated,1.0,-2.226913,-2.087215,not participated,1.0,.95921,-.07942,not participated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,1,13.0,13.0,1,71.550003,36.349998,1,2460.0,2636.0,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
3477,8069741,1,13.0,13.0,1,25.040001,24.530001,1,5123.0,5131.0,...,not participated,not participated,1.0,-1.813545,not participated,not participated,1.0,not participated,not participated,not participated
3478,8069742,1,18.0,18.0,1,82.410004,70.570000,1,2320.0,2330.0,...,9,not participated,1.0,1.903898,.3931048,1.6124,1.0,.78287,.75734,not participated
3479,8069744,1,15.0,13.0,1,44.939999,28.480000,1,4115.0,4120.0,...,8,7,1.0,-.9448132,-.6620925,-.9287,1.0,-.71679,-2.01072,-.34228


In [35]:
# Conversion of the type of column
neps['mISCO_08'] = neps['mISCO_08'].astype(str)
neps['fISCO_08'] = neps['fISCO_08'].astype(str)

# We make compatible the formats of ISCO code
neps['mISCO_08'] = neps['mISCO_08'].str[:4]
neps['fISCO_08'] = neps['fISCO_08'].str[:4]

,mISCO_08,fISCO_08
0,2100,3341
1,2432,2512
2,2635,3343
3,5120,5120
4,5142,1420
...,...,...
3476,2636,7411
3477,5131,5120
3478,2330,1213
3479,4120,5223


In [36]:
job_skill['isco08'] = job_skill['isco08'].astype(str) #We convert the type in the job_skill database to allow the merge

In [37]:
#We merge to have, for each children outcome, the skills of their mothers. 
merge_mother = pd.merge(neps, job_skill, left_on='mISCO_08', right_on ='isco08', how='left')
merge_mother

,ID_t,PARENTS_EDUCATION,meduYRS,feduYRS,PARENTS_ISEI,mISEI,fISEI,PARENTS_ISCO,mISCO_88,mISCO_08,...,scn10_sc1,isco08,IM_cog_deci_skills,IM_manual_skills,IM_comp_skills,IM_manag_skills,LV_cog_deci_skills,LV_manual_skills,LV_comp_skills,LV_manag_skills
0,8054956,1,16.0,16.0,1,79.489998,62.130001,1,2140.0,2100,...,.72872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8054957,1,18.0,16.0,1,73.910004,74.660004,1,2419.0,2432,...,not participated,2432,1.086538,-1.342790,-0.162715,-0.212655,1.025959,-1.340247,-0.317215,0.052874
2,8054966,1,16.0,13.0,1,70.500000,54.549999,1,2446.0,2635,...,-.21941,2635,1.977610,-0.879875,-0.261480,-1.425886,2.018624,-0.896666,-0.955235,-1.785156
3,8054975,1,13.0,13.0,1,24.530001,24.530001,1,5122.0,5120,...,.43187,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
4,8054979,1,15.0,15.0,1,31.080000,51.560001,1,5141.0,5142,...,not participated,5142,-0.550591,-0.947607,-0.679031,-0.091265,-0.759187,-0.977561,-0.316722,0.035603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,1,13.0,13.0,1,71.550003,36.349998,1,2460.0,2636,...,NaN,2636,1.452434,-1.079344,-0.519187,1.018556,1.650983,-0.903388,-1.357376,0.970316
3477,8069741,1,13.0,13.0,1,25.040001,24.530001,1,5123.0,5131,...,not participated,5131,-0.965300,-0.972108,-1.180777,0.060780,-1.641216,-1.071598,-0.385629,0.074008
3478,8069742,1,18.0,18.0,1,82.410004,70.570000,1,2320.0,2330,...,not participated,2330,1.885723,-0.654637,0.068545,-1.352704,1.493149,-0.831712,-0.458447,-1.263053
3479,8069744,1,15.0,13.0,1,44.939999,28.480000,1,4115.0,4120,...,-.34228,4120,-0.388678,-1.396197,-0.423427,-0.413182,-0.497420,-1.373454,-0.057375,-0.220687


In [38]:
#We rename the wolumns in order to specify that we talk about mother's skills
merge_mother.rename(columns={ 'IM_cog_deci_skills' : 'm_IMCogDeciS', 'IM_manual_skills' : 'm_IMManualS',	'IM_comp_skills' : 'm_IMCoS',	'IM_manag_skills' : 'm_IMManagS',	'LV_cog_deci_skills' : 'm_LVCogDeciS',	'LV_manual_skills' : 'm_LVManualS',	'LV_comp_skills' : 'm_LVCoS' ,	'LV_manag_skills' : 'm_LVManagS'}, inplace=True)
merge_mother

,ID_t,PARENTS_EDUCATION,meduYRS,feduYRS,PARENTS_ISEI,mISEI,fISEI,PARENTS_ISCO,mISCO_88,mISCO_08,...,scn10_sc1,isco08,m_IMCogDeciS,m_IMManualS,m_IMCoS,m_IMManagS,m_LVCogDeciS,m_LVManualS,m_LVCoS,m_LVManagS
0,8054956,1,16.0,16.0,1,79.489998,62.130001,1,2140.0,2100,...,.72872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8054957,1,18.0,16.0,1,73.910004,74.660004,1,2419.0,2432,...,not participated,2432,1.086538,-1.342790,-0.162715,-0.212655,1.025959,-1.340247,-0.317215,0.052874
2,8054966,1,16.0,13.0,1,70.500000,54.549999,1,2446.0,2635,...,-.21941,2635,1.977610,-0.879875,-0.261480,-1.425886,2.018624,-0.896666,-0.955235,-1.785156
3,8054975,1,13.0,13.0,1,24.530001,24.530001,1,5122.0,5120,...,.43187,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
4,8054979,1,15.0,15.0,1,31.080000,51.560001,1,5141.0,5142,...,not participated,5142,-0.550591,-0.947607,-0.679031,-0.091265,-0.759187,-0.977561,-0.316722,0.035603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,1,13.0,13.0,1,71.550003,36.349998,1,2460.0,2636,...,NaN,2636,1.452434,-1.079344,-0.519187,1.018556,1.650983,-0.903388,-1.357376,0.970316
3477,8069741,1,13.0,13.0,1,25.040001,24.530001,1,5123.0,5131,...,not participated,5131,-0.965300,-0.972108,-1.180777,0.060780,-1.641216,-1.071598,-0.385629,0.074008
3478,8069742,1,18.0,18.0,1,82.410004,70.570000,1,2320.0,2330,...,not participated,2330,1.885723,-0.654637,0.068545,-1.352704,1.493149,-0.831712,-0.458447,-1.263053
3479,8069744,1,15.0,13.0,1,44.939999,28.480000,1,4115.0,4120,...,-.34228,4120,-0.388678,-1.396197,-0.423427,-0.413182,-0.497420,-1.373454,-0.057375,-0.220687


In [40]:
#We add father characteristics for each children
father_merge = pd.merge(merge_mother, job_skill, left_on='fISCO_08', right_on='isco08', how='left')
father_merge

,ID_t,PARENTS_EDUCATION,meduYRS,feduYRS,PARENTS_ISEI,mISEI,fISEI,PARENTS_ISCO,mISCO_88,mISCO_08,...,m_LVManagS,isco08_y,IM_cog_deci_skills,IM_manual_skills,IM_comp_skills,IM_manag_skills,LV_cog_deci_skills,LV_manual_skills,LV_comp_skills,LV_manag_skills
0,8054956,1,16.0,16.0,1,79.489998,62.130001,1,2140.0,2100,...,NaN,3341,1.824293,-0.397477,-0.703958,1.517878,0.898988,-0.549374,-0.755013,1.547815
1,8054957,1,18.0,16.0,1,73.910004,74.660004,1,2419.0,2432,...,0.052874,2512,-0.583782,0.249249,3.276386,-1.639028,-0.032607,0.195047,3.534941,-1.681438
2,8054966,1,16.0,13.0,1,70.500000,54.549999,1,2446.0,2635,...,-1.785156,3343,-0.742728,-1.112078,-0.233284,-1.156076,-0.666041,-1.104842,0.140653,-1.438003
3,8054975,1,13.0,13.0,1,24.530001,24.530001,1,5122.0,5120,...,1.706339,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
4,8054979,1,15.0,15.0,1,31.080000,51.560001,1,5141.0,5142,...,0.035603,1420,1.025919,-0.296045,-0.523645,2.746239,0.780411,-0.357413,-0.400647,2.322201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,1,13.0,13.0,1,71.550003,36.349998,1,2460.0,2636,...,0.970316,7411,0.266288,1.937660,0.265476,-0.121493,0.484758,2.255369,0.121894,-0.326734
3477,8069741,1,13.0,13.0,1,25.040001,24.530001,1,5123.0,5131,...,0.074008,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
3478,8069742,1,18.0,18.0,1,82.410004,70.570000,1,2320.0,2330,...,-1.263053,1213,0.987199,-0.322260,0.057986,1.295678,0.974806,-0.296031,-0.274092,1.350375
3479,8069744,1,15.0,13.0,1,44.939999,28.480000,1,4115.0,4120,...,-0.220687,5223,0.286365,-0.630684,-0.938806,-0.099073,-0.091834,-0.516989,-0.788416,-0.317386


In [41]:
#We rename the columns in order to specify that we talk about father's skills
father_merge.rename(columns={ 'IM_cog_deci_skills' : 'f_IMCogDeciS', 'IM_manual_skills' : 'f_IMManualS',	'IM_comp_skills' : 'f_IMCoS',	'IM_manag_skills' : 'f_IMManagS',	'LV_cog_deci_skills' : 'f_LVCogDeciS',	'LV_manual_skills' : 'f_LVManualS',	'LV_comp_skills' : 'f_LVCoS' ,	'LV_manag_skills' : 'f_LVManagS'}, inplace=True)
father_merge

,ID_t,PARENTS_EDUCATION,meduYRS,feduYRS,PARENTS_ISEI,mISEI,fISEI,PARENTS_ISCO,mISCO_88,mISCO_08,...,m_LVManagS,isco08_y,f_IMCogDeciS,f_IMManualS,f_IMCoS,f_IMManagS,f_LVCogDeciS,f_LVManualS,f_LVCoS,f_LVManagS
0,8054956,1,16.0,16.0,1,79.489998,62.130001,1,2140.0,2100,...,NaN,3341,1.824293,-0.397477,-0.703958,1.517878,0.898988,-0.549374,-0.755013,1.547815
1,8054957,1,18.0,16.0,1,73.910004,74.660004,1,2419.0,2432,...,0.052874,2512,-0.583782,0.249249,3.276386,-1.639028,-0.032607,0.195047,3.534941,-1.681438
2,8054966,1,16.0,13.0,1,70.500000,54.549999,1,2446.0,2635,...,-1.785156,3343,-0.742728,-1.112078,-0.233284,-1.156076,-0.666041,-1.104842,0.140653,-1.438003
3,8054975,1,13.0,13.0,1,24.530001,24.530001,1,5122.0,5120,...,1.706339,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
4,8054979,1,15.0,15.0,1,31.080000,51.560001,1,5141.0,5142,...,0.035603,1420,1.025919,-0.296045,-0.523645,2.746239,0.780411,-0.357413,-0.400647,2.322201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,1,13.0,13.0,1,71.550003,36.349998,1,2460.0,2636,...,0.970316,7411,0.266288,1.937660,0.265476,-0.121493,0.484758,2.255369,0.121894,-0.326734
3477,8069741,1,13.0,13.0,1,25.040001,24.530001,1,5123.0,5131,...,0.074008,5120,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
3478,8069742,1,18.0,18.0,1,82.410004,70.570000,1,2320.0,2330,...,-1.263053,1213,0.987199,-0.322260,0.057986,1.295678,0.974806,-0.296031,-0.274092,1.350375
3479,8069744,1,15.0,13.0,1,44.939999,28.480000,1,4115.0,4120,...,-0.220687,5223,0.286365,-0.630684,-0.938806,-0.099073,-0.091834,-0.516989,-0.788416,-0.317386


In [42]:
father_merge.columns

Index(['ID_t', 'PARENTS_EDUCATION', 'meduYRS', 'feduYRS', 'PARENTS_ISEI',
       'mISEI', 'fISEI', 'PARENTS_ISCO', 'mISCO_88', 'mISCO_08', 'fISCO_88',
       'fISCO_08', 'CHILD_SKILLS', 'COGNITIVE', 'cdn1_sc1', 'COGNITIVE_VOCAB',
       'von4_sc3', 'von6_sc3', 'von8_sc3', 'von10_sc3', 'COGNITIVE_DIGIT',
       'dsn40001_sc3a', 'dsn70001_sc3a', 'bdn80001_sc3a', 'COGNITIVE_BASIC',
       'dgn7_sc3a', 'dgn7_sc3b', 'dgn10_sc3b', 'COGNITIVE_MATH', 'man5_sc1',
       'man7_sc1', 'man9_sc1', 'COGNITIVE_SCIENCE', 'scn6_sc1', 'scn8_sc1',
       'scn10_sc1', 'isco08_x', 'm_IMCogDeciS', 'm_IMManualS', 'm_IMCoS',
       'm_IMManagS', 'm_LVCogDeciS', 'm_LVManualS', 'm_LVCoS', 'm_LVManagS',
       'isco08_y', 'f_IMCogDeciS', 'f_IMManualS', 'f_IMCoS', 'f_IMManagS',
       'f_LVCogDeciS', 'f_LVManualS', 'f_LVCoS', 'f_LVManagS'],
      dtype='object')

In [44]:
#We delete useless columns
df = father_merge.drop(['PARENTS_EDUCATION','PARENTS_ISEI', 'PARENTS_ISCO', 'mISCO_88', 'fISCO_88', 'CHILD_SKILLS', 'COGNITIVE','COGNITIVE_VOCAB', 'COGNITIVE_DIGIT', 'COGNITIVE_BASIC','COGNITIVE_MATH',  'COGNITIVE_SCIENCE','isco08_x','isco08_y'], axis=1, inplace=False)
df

,ID_t,meduYRS,feduYRS,mISEI,fISEI,mISCO_08,fISCO_08,cdn1_sc1,von4_sc3,von6_sc3,...,m_LVCoS,m_LVManagS,f_IMCogDeciS,f_IMManualS,f_IMCoS,f_IMManagS,f_LVCogDeciS,f_LVManualS,f_LVCoS,f_LVManagS
0,8054956,16.0,16.0,79.489998,62.130001,2100,3341,2.01033,Practice phase not passed,93,...,NaN,NaN,1.824293,-0.397477,-0.703958,1.517878,0.898988,-0.549374,-0.755013,1.547815
1,8054957,18.0,16.0,73.910004,74.660004,2432,2512,.l,Practice phase not passed,34,...,-0.317215,0.052874,-0.583782,0.249249,3.276386,-1.639028,-0.032607,0.195047,3.534941,-1.681438
2,8054966,16.0,13.0,70.500000,54.549999,2635,3343,-.36671,25,47,...,-0.955235,-1.785156,-0.742728,-1.112078,-0.233284,-1.156076,-0.666041,-1.104842,0.140653,-1.438003
3,8054975,13.0,13.0,24.530001,24.530001,5120,5120,1.01754,No valid statement possible: no response in al...,71,...,-0.467504,1.706339,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
4,8054979,15.0,15.0,31.080000,51.560001,5142,1420,2.22342,3,92,...,-0.316722,0.035603,1.025919,-0.296045,-0.523645,2.746239,0.780411,-0.357413,-0.400647,2.322201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3476,8069729,13.0,13.0,71.550003,36.349998,2636,7411,.17971,NaN,NaN,...,-1.357376,0.970316,0.266288,1.937660,0.265476,-0.121493,0.484758,2.255369,0.121894,-0.326734
3477,8069741,13.0,13.0,25.040001,24.530001,5131,5120,-.79319,not participated,not participated,...,-0.385629,0.074008,-0.945854,-0.446603,-0.652797,1.873571,-0.795232,-0.545063,-0.467504,1.706339
3478,8069742,18.0,18.0,82.410004,70.570000,2330,1213,-3.57539,37,87,...,-0.458447,-1.263053,0.987199,-0.322260,0.057986,1.295678,0.974806,-0.296031,-0.274092,1.350375
3479,8069744,15.0,13.0,44.939999,28.480000,4120,5223,-.92494,No valid statement possible: no response in al...,63,...,-0.057375,-0.220687,0.286365,-0.630684,-0.938806,-0.099073,-0.091834,-0.516989,-0.788416,-0.317386


In [45]:
#We export the database 'I didn't share it on github because I'm not allowed to
df.to_stata('base_def.dta')
df.to_excel('base_def.xlsx')